In [5]:
# !pip install PyPDF2

In [6]:
## Getting to Main directory
import os
os.chdir("../")

In [2]:

import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [7]:
load_dotenv()
gemini_api_key=os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [8]:
pwd

'e:\\Personal Projects\\legal-agent'

In [9]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('resources', glob="**/*.pdf")
docs = loader.load()
len(docs)

c:\Users\Rajarshi Roy\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


2

In [10]:
docs

[Document(metadata={'source': 'resources\\Guide-to-Litigation-in-India.pdf'}, page_content='Guide to Litigation in India\n\nA Cyril Amarchand Mangaldas Thought Leadership Publication\n\nGuide to Litigation in India is published by Cyril Amarchand Mangaldas.\n\nThis guide has been updated till April, 2023.\n\nThis publication is not intended to be used as the basis for trading in the shares of any company or for undertaking any other complex or significant financial transaction without consulting appropriate professional advisers. No part of this publication may be copied or redistributed in any form without the prior written consent of Cyril Amarchand Mangaldas.\n\nCopyright © 2023 Cyril Amarchand Mangaldas. All rights reserved.\n\nA Thought Leadership Publication\n\nWe now present this guide to enable readers to have an overview of the systems and legal rules and regulations that are essential for business operations in India.\n\nGuide to Litigation in India\n\nTABLE OF CONTENTS\n\nIN

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
text_chunks = text_splitter.split_documents(docs)
len(text_chunks)

538

In [12]:
# Clean up our Documents' content
import re

def clean_up_text(content: str) -> str:
    """
    Remove unwanted characters and patterns in text input.

    :param content: Text input.
    
    :return: Cleaned version of original text input.
    """

    # Fix hyphenated words broken by newline
    content = re.sub(r'(\w+)-\n(\w+)', r'\1\2', content)

    # Remove specific unwanted patterns and characters
    unwanted_patterns = [
        "\\n", "  —", "——————————", "—————————", "—————",
        r'\\u[\dA-Fa-f]{4}', r'\uf075', r'\uf0b7'
    ]
    for pattern in unwanted_patterns:
        content = re.sub(pattern, "", content)

    # Fix improperly spaced hyphenated words and normalize whitespace
    content = re.sub(r'(\w)\s*-\s*(\w)', r'\1-\2', content)
    content = re.sub(r'\s+', ' ', content)

    return content

# Call function
cleaned_docs = []
for d in text_chunks: 
    cleaned_text = clean_up_text(d.page_content)
    d.page_content = cleaned_text
    cleaned_docs.append(d)

'Guide to Litigation in IndiaA Cyril Amarchand Mangaldas Thought Leadership PublicationGuide to Litigation in India is published by Cyril Amarchand Mangaldas.This guide has been updated till April, 2023.This publication is not intended to be used as the basis for trading in the shares of any company or for undertaking any other complex or significant financial transaction without consulting appropriate professional advisers. No part of this publication may be copied or redistributed in any form without the prior written consent of Cyril Amarchand Mangaldas.Copyright © 2023 Cyril Amarchand Mangaldas. All rights reserved.A Thought Leadership PublicationWe now present this guide to enable readers to have an overview of the systems and legal rules and regulations that are essential for business operations in India.Guide to Litigation in IndiaTABLE OF CONTENTSINTRODUCTIONA. LITIGATION IN CIVIL COURTS1.India-A Common Law System2.India’s Judicial System'

In [15]:
cleaned_docs[0].page_content

'Guide to Litigation in IndiaA Cyril Amarchand Mangaldas Thought Leadership PublicationGuide to Litigation in India is published by Cyril Amarchand Mangaldas.This guide has been updated till April, 2023.This publication is not intended to be used as the basis for trading in the shares of any company or for undertaking any other complex or significant financial transaction without consulting appropriate professional advisers. No part of this publication may be copied or redistributed in any form without the prior written consent of Cyril Amarchand Mangaldas.Copyright © 2023 Cyril Amarchand Mangaldas. All rights reserved.A Thought Leadership PublicationWe now present this guide to enable readers to have an overview of the systems and legal rules and regulations that are essential for business operations in India.Guide to Litigation in IndiaTABLE OF CONTENTSINTRODUCTIONA. LITIGATION IN CIVIL COURTS1.India-A Common Law System2.India’s Judicial System'

In [16]:
cleaned_docs[0].metadata

{'source': 'resources\\Guide-to-Litigation-in-India.pdf'}

In [17]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
genai.configure(api_key=gemini_api_key)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# Setting Tempreture to 0.3 for getting low risk results

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",api_key=gemini_api_key,temperature=0.3)
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
vector_store.save_local("faiss_index")

In [4]:
import faiss

index = faiss.read_index("../faiss_index/index.faiss")
print("Index type:", type(index))


Index type: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>


In [1]:
import faiss

print("Index type:", type(faiss_index))
# A simple check based on the class name:
if "Gpu" in str(type(faiss_index)):
    print("This is a GPU index.")
else:
    print("This is a CPU index.")


NameError: name 'faiss_index' is not defined